In [108]:
from PIL import Image
from Cryptodome.Cipher import AES, Salsa20
# from pycryto.Cipher import AES
# from Crypto.Util.Padding import pad, unpad
from Cryptodome.Random import get_random_bytes
from Cryptodome.Util.Padding import pad, unpad


In [2]:
def string_to_binary(message):
    encodeM = str(message).encode()
    bin_int = int.from_bytes(encodeM, 'big')
    bin_string = bin(bin_int)[2:]
    return "0" + bin_string

In [3]:
def encode_message(pixels, binString):
    # loop through each pixel
    for i in range(len(pixels)):
        for j in range(len(pixels[i])):
            if (binString == ''):
                return pixels
            # get the pixel value this is in binary already
            pixel = pixels[i][j]
            # print(pixel)
            # get first two characters of binString to be implanted into pixel
            binStringC = binString[:2]
            # remove first two characters of binString to the message will be eaten while looping
            binString = binString[2:]
            # get the first 6 character of binC then add the two bits of binStringC to the end
            pixel = pixel[:6] + binStringC
            #set the new pixel value
            pixels[i][j] = pixel
    # return pixels


 
                

In [4]:
def decode_message(pixelsWithMessage):
    binaryPixels = [list(bin(p)[2:].rjust(8,'0') for p in pixel) for pixel in pixelsWithMessage]
    binPixels = [list(pixel[-2:] for pixel in pixel) for pixel in binaryPixels]
    # binPixels
    # iterate through each binPixels and concatenate them into a string
    binString = ''.join([''.join(pixel) for pixel in binPixels])
    #split into 8 bit chunks
    binString = [binString[i:i+8] for i in range(0, len(binString), 8)]
    # print(binString)

    # iterate through each binString and convert to int
    binString = [int(b, 2) for b in binString]
    # binString
    # turn ascii numbers to ascii characters
    binString = [chr(b) for b in binString]

    binString = ''.join(binString)
    return binString

In [5]:
def binary_to_string(binString):
    binary_int = int(binString, 2)  
    byte_number = binary_int.bit_length() + 7 // 8
    binary_array = binary_int.to_bytes(byte_number, "big")
    ascii_text = binary_array.decode()
    return ascii_text

In [6]:
def canEncode(pixels, message):
    LengthM = len(string_to_binary(message))
    LengthP = len(pixels) *3 *2
    print("message length: ", LengthM)
    print("pixels length: ", LengthP)
    if LengthM > LengthP:
        return False, LengthM, LengthP
    return True, LengthM, LengthP

# Hide info and create photo

In [8]:
def hide_Message(Photo1, hiddenMessage, photoName):
    # turn txt file to binary
    binMessage = string_to_binary(hiddenMessage)
    # turn photo to pixel representation
    pixels = Photo1.getdata()
    # turn pixel representation into a list of lists
    binaryPixels = [list(bin(p)[2:].rjust(8,'0') for p in pixel) for pixel in pixels]
    # implant the message
    newPhotoPixelsBinary = encode_message(binaryPixels, binMessage)
    # turn the binary pixels back into rgb values
    newPhotoPixels = [tuple(int(p,2) for p in pixel) for pixel in newPhotoPixelsBinary]
    # turn the rgb values back into a photo
    newPhoto = Image.new("RGB", Photo1.size)
    newPhoto.putdata(newPhotoPixels)
    # save the photo
    finalFilename = ".".join([photoName, "png"])
    newPhoto.save(finalFilename)



In [87]:
Photo1.size

(1700, 744)

In [88]:
pixelsWithMessage.size

(1700, 744)

# examples

In [109]:
# decrypt_test

## no encyrption

In [9]:
Photo1 = Image.open("startImages/Photo1.jpg")
Photo2 = Image.open('startImages/Photo2.jpg')

hiddenMessage = open('ThingsToHide/test.txt', 'r').read()
hiddenMessage = str(hiddenMessage)
# testB = string_to_binary(hiddenMessage)

In [10]:
if canEncode(Photo1.getdata(), hiddenMessage):
    hide_Message(Photo1, hiddenMessage, "encodedImages/notEncypted")

message length:  110440
pixels length:  7588800


In [11]:
notEncryptedMessage = Image.open('encodedImages/notEncypted.png')
pixelsWithMessage = notEncryptedMessage.getdata()
LBSMessage = decode_message(pixelsWithMessage)
# LBSMessage
# save the message to a file
# with open('decodedMessages/LSBMessage_noEncryption.txt', 'w') as f:
#     f.write(LBSMessage)


# with encryption

In [12]:
Photo1 = Image.open("startImages/Photo1.jpg")
hiddenMessage = open('ThingsToHide/test.txt', 'r').read()
hiddenMessage = str(hiddenMessage)

In [107]:
#this works can not decrpyt from it
data = bytes(hiddenMessage, 'utf-8')
key = b'\x14\xdaE\xee-Tn\xa3\xf6\xce<\x05,\x00\xadF'
print(" this is the key: ", key)
cipher = AES.new(key, AES.MODE_ECB)
padded_text = pad(data, 16)
# print(padded_text)
encrypted_message = cipher.encrypt(padded_text)
len(encrypted_message)
type(encrypted_message)
# test = unpad(padded_text, 16)
# test

 this is the key:  b'\x14\xdaE\xee-Tn\xa3\xf6\xce<\x05,\x00\xadF'


bytes

test to make sure decrypts correctly

In [99]:
decipher = AES.new(key, AES.MODE_ECB)
decrypted_message = decipher.decrypt(encrypted_message)
unpadded_decrypted_message = unpad(decrypted_message, 16)   

# print(unpadded_decrypted_message)

In [100]:
# hide the message in the photo
if canEncode(Photo1.getdata(), str(encrypted_message)):
    hide_Message(Photo1, str(encrypted_message), "encodedImages/encrypted")

message length:  316744
pixels length:  7588800


In [101]:
hiddenImgEncrypted = Image.open('encodedImages/encrypted.png')

# with open("encodedImages/encrypted.png", "rb") as image:
#   f = image.read()
#   b = bytearray(f)
# print(b)
# pixelsWithMessageEncrypted = b
pixelsWithMessageEncrypted = hiddenImgEncrypted.getdata()

In [102]:
print(len(pixelsWithMessageEncrypted))
print(pixelsWithMessageEncrypted[0])
print(1700*744*3*2)

1264800
(109, 66, 60)
7588800


In [103]:
encrypted_message_from_Img = decode_message(pixelsWithMessageEncrypted)#decode

In [104]:
print((len(encrypted_message)))
print(len(encrypted_message_from_Img))

13808
948600


In [105]:

encrypted_bytes = bytes(encrypted_message_from_Img, 'utf-8')
encrypted_bytes = encrypted_bytes[:13808]
type(encrypted_message)
decipher = AES.new(key, AES.MODE_ECB)

decrypted_message = decipher.decrypt(encrypted_bytes)
# decrypted_test = decipher.decrypt(decrypt_bytes)


# decrypted_test
dectpt_unpad= unpad(decrypted_message, 16)
# decrypted_test = decipher.decrypt(pad(encrypted_test,16))
# decrypted_test = unpad(decrypted_test, 16)
# decrypted_test = decrypted_test.decode('utf-8')
# print(decrypted_test)

ValueError: PKCS#7 padding is incorrect.

# salsa20

In [109]:
Photo1 = Image.open("startImages/Photo1.jpg")
hiddenMessage = open('ThingsToHide/test.txt', 'r').read()
# hiddenMessage = str(hiddenMessage)

In [118]:
keyS = b'\x14\xdaE\xee-Tn\xa3\xf6\xce<\x05,\x00\xadF\x14\xdaE\xee-Tn\xa3\xf6\xce<\x05,\x00\xadF'
Scipher = Salsa20.new(key = keyS)
enc_msg = Scipher.nonce + Scipher.encrypt(b'hiddenMessage')
# enc_msg = 

In [119]:
if canEncode(Photo1.getdata(), str(enc_msg)):
    hide_Message(Photo1, str(enc_msg), "encodedImages/encrypted")

message length:  432
pixels length:  7588800


In [121]:
hiddenImgEncrypted = Image.open('encodedImages/encrypted.png')
pixelsWithMessageEncrypted = hiddenImgEncrypted.getdata()

In [128]:
encrypted_message_from_Img = decode_message(pixelsWithMessageEncrypted)#decode


TypeError: string argument without an encoding

In [131]:
encrypted_message_from_Img = bytes(encrypted_message_from_Img, 'utf-8')

In [132]:
msg_nonce = encrypted_message_from_Img[:8]
# msg_nonce = bytes(msg_nonce)
cipher_text = encrypted_message_from_Img[8:]
cipher = Salsa20.new(key = keyS, nonce = msg_nonce)
hope = cipher.decrypt(cipher_text)

In [137]:
message = decode_message(hope)

TypeError: 'int' object is not iterable